In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pywt

In [2]:
import pickle as pic
from itertools import product

In [3]:
from utils import *
from templates import *
from signal_strength import SIGNAL_STRENGTHS_METHODS
from data_loader import EntireTraceIterator

In [4]:
with open("wavelets_haar_2.pic", "rb") as r:
    wavelets = pic.load(r)
with open("labels.pic", "rb") as r:
    rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

In [4]:
traces_path = "..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.mat"
key_path = "..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.log"

In [5]:
NUM_TRACES = 1_000_000
TRACE_SIZE = 200_000
PER_DIVISION = 250_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=1, nr_scenarios=1, trace_size=TRACE_SIZE, traces_per_division=PER_DIVISION, parse_output="keyshares+perms")

In [4]:
if True:
    with open("orig_first_250000.pic", "rb") as r:
        traces = pic.load(r)
else:    
    for seeds, traces, key, output in data_loader((0,), (0,)):
        seeds = seeds[0][0]
        traces = traces[0][0]
        key = np.array(key[0])
        output = output[0][0]
        break
with open("labels_1000000.pic", "rb") as r:
    rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

In [5]:
NUM_TRACES = 1_000_000
TRACE_SIZE = 200_000
PER_DIVISION = 1_000_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=1, nr_scenarios=1, trace_size=TRACE_SIZE, traces_per_division=PER_DIVISION, parse_output="keyshares+perms")

In [6]:
for seeds, key, _ in data_loader((0,), (0,), return_traces=False):
    seeds = seeds[0][0]
    key = np.array(key[0])
    break

with open("seeds_key_1000000.pic", "wb") as w:
    pic.dump((seeds, key), w)

## Wavelets

In [ ]:
for j, (_, traces, _, _) in enumerate(data_loader((0,), (0,))):
    traces = traces[0][0]
    for i in range(0, PER_DIVISION, PER_DIVISION // 10):
        wavelet = pywt.wavedec(traces[i:i + PER_DIVISION // 10], 'haar', level=2)[0]
        with open(f"./wavelets_haar_2/wavelet_{(j+2) * PER_DIVISION + i}_{(j+2) * PER_DIVISION + i + PER_DIVISION // 10}.pic", "wb") as w:
            pic.dump(wavelet, w)
    del traces

In [9]:
wavelets = np.zeros((300_000, 50000), dtype=np.float16)
for div in range(0, 315000, 35000):
    filepath = f"./wavelets_haar_2/wavelet_{700_000 + div}_{700_000 + div + 35000}.pic"
    with open(filepath, "rb") as r:
        wavelet = pic.load(r)
    wavelets[div:div+wavelet.shape[0]] = np.float16(wavelet)

In [11]:
np.where(np.all(wavelets == 0, axis=1))

(array([499999], dtype=int64),)

In [12]:
wavelets = np.concatenate((wavelets[:499999], wavelets[500000:]), axis=0)


In [ ]:
with open("wavelets_haar_2.pic", "rb") as r:
    first_750000 = pic.load(r)

In [ ]:
np.concatenate((first_750000, wavelets)).shape

In [13]:
with open("wavelets_haar_2.pic", "wb") as w:
    pic.dump(wavelets, w)

## Labels

In [6]:
rws_perms_labels = np.zeros(NUM_TRACES, dtype=int)
rws_masks_labels = np.zeros((KEYROUND_WIDTH_B4, NR_SHARES, NUM_TRACES), dtype=int)
round_perms_labels = np.zeros(NUM_TRACES, dtype=int)
copy_perms_labels = np.zeros((LATEST_ROUND - EARLIEST_ROUND, NUM_TRACES), dtype=int)
round_masks_labels = np.zeros((LATEST_ROUND - EARLIEST_ROUND, BLOCK_WIDTH_B4, NR_SHARES, NUM_TRACES), dtype=int)

for j, (seeds_sub, key, output_sub) in enumerate(data_loader((0,), (0,), return_traces=False)):
    rws_perms, rws_masks, round_perms, copy_perms, round_masks = get_all_labels(seeds_sub[0][0], key[0], output_sub[0][0])
    rws_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + rws_perms.shape[-1]] = rws_perms
    rws_masks_labels[..., j * PER_DIVISION:j * PER_DIVISION + rws_masks.shape[-1]] = rws_masks
    round_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + round_perms.shape[-1]] = round_perms
    copy_perms_labels[..., j * PER_DIVISION:j * PER_DIVISION + copy_perms.shape[-1]] = copy_perms
    round_masks_labels[..., j * PER_DIVISION:j * PER_DIVISION + round_masks.shape[-1]] = round_masks

In [7]:
data_loader.empty_traces

[324360]

In [10]:
rws_perms_labels = np.concatenate((rws_perms_labels[..., :499999], rws_perms_labels[..., 500000:]), axis=-1)
rws_masks_labels = np.concatenate((rws_masks_labels[..., :499999], rws_masks_labels[..., 500000:]), axis=-1)
round_perms_labels = np.concatenate((round_perms_labels[..., :499999], round_perms_labels[..., 500000:]), axis=-1)
copy_perms_labels = np.concatenate((copy_perms_labels[..., :499999], copy_perms_labels[..., 500000:]), axis=-1)
round_masks_labels = np.concatenate((round_masks_labels[..., :499999], round_masks_labels[..., 500000:]), axis=-1)

In [11]:
with open("labels_1000000.pic", "wb") as w:
    pic.dump((rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels), w)

## RWS permutations

In [7]:
signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("./leakage_points_bigdata/f_rws_perms_sost.pic")
signal_strength.fit(traces, rws_perms_labels[:PER_DIVISION], 0)

## RWS masks

In [11]:
for keyround_idx in range(KEYROUND_WIDTH_B4 // 3):
    first = keyround_idx + KEYROUND_WIDTH_B4 // 4 - 1
    print(f"{first}", end="\r")
    for share_idx in range(NR_SHARES):
        signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_bigdata/f_rws_masks_{first}_{share_idx}_sost.pic")
        signal_strength.fit(traces, rws_masks_labels[first, share_idx, :PER_DIVISION], 0)
    second = first + KEYROUND_WIDTH_B4 // 4
    print(f"{second}", end="\r")
    for share_idx in range(NR_SHARES):
        signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_bigdata/f_rws_masks_{second}_{share_idx}_sost.pic")
        signal_strength.fit(traces, rws_masks_labels[second, share_idx, :PER_DIVISION], 0)
    third = second + KEYROUND_WIDTH_B4 // 4
    print(f"{third}", end="\r")
    for share_idx in range(NR_SHARES):
        signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_bigdata/f_rws_masks_{third}_{share_idx}_sost.pic")
        signal_strength.fit(traces, rws_masks_labels[third, share_idx, :PER_DIVISION], 0)

KeyboardInterrupt: 

## Round permutations

In [8]:
signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"]("./leakage_points_bigdata/f_round_perms_sost.pic")
signal_strength.fit(traces, round_perms_labels[:PER_DIVISION], 0)

## Copy permutations

In [9]:
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    print(f"{round_idx}", end="\r")
    signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_bigdata/f_copy_perms_{round_idx}_sost.pic")
    signal_strength.fit(traces, copy_perms_labels[round_idx, :PER_DIVISION], 0)

## Round masks

In [6]:
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    for block_idx in range(BLOCK_WIDTH_B4):
        print(f"{round_idx * BLOCK_WIDTH_B4 + block_idx}", end="\r")
        for share_idx in range(NR_SHARES):
            signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_bigdata/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic")
            signal_strength.fit(traces, round_masks_labels[round_idx, block_idx, share_idx, :PER_DIVISION], 0)

In [ ]:
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    for block_idx in range(BLOCK_WIDTH_B4):
        print(f"{round_idx * BLOCK_WIDTH_B4 + block_idx}", end="\r")
        for share_idx in range(NR_SHARES):
            signal_strength = SIGNAL_STRENGTHS_METHODS["SOST"](f"./leakage_points_haar_2/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic")
            signal_strength.fit(wavelets, round_masks_labels[round_idx, block_idx, share_idx], 0)

## Smaller windows

In [6]:
RWS_WINDOW = np.arange(1100, 21500)
ROUND_PERM_WINDOW = np.concatenate((
        np.arange( 21750,  21800),
        np.arange( 33925,  33975),
        np.arange( 46250,  46300),
        np.arange( 58550,  25600),
        np.arange( 70860,  70910),
        np.arange( 83180,  83230),
        np.arange( 95480,  95530),
        np.arange(107800, 107850),
        np.arange(120110, 120160),
        np.arange(132415, 132465),
        np.arange(144725, 144775),
        np.arange(157040, 157090),
        np.arange(169350, 169400),
        np.arange(181660, 181710),
        np.arange(193960, 194010),
    ), axis=0)
ROUND_WINDOWS = np.stack((
    np.arange( 21950,  23350), # Round 0
    np.arange( 34250,  35650), # Round 1
    np.arange( 46550,  47950), # Round 2
    np.arange( 58850,  60250), # Round 3
    np.arange( 71150,  72550), # Round 4
    np.arange( 83500,  84900), # Round 5
    np.arange( 95800,  97200), # Round 6
    np.arange(108100, 109500), # Round 7
    np.arange(120400, 121800), # Round 8
    np.arange(132700, 134100), # Round 9
    np.arange(145025, 146425), # Round 10
    np.arange(157350, 158750), # Round 11
    np.arange(169650, 171050), # Round 12
    np.arange(181950, 183350), # Round 13
), axis=0)
#traces_rws_only = np.zeros((NUM_TRACES, len(RWS_WINDOW)), dtype=np.float16)
#traces_round_perm_only = np.zeros((NUM_TRACES, len(ROUND_PERM_WINDOW)), dtype=np.float16)
traces_rounds_i_only = np.zeros((ROUND_WINDOWS.shape[0], NUM_TRACES, ROUND_WINDOWS.shape[1]), dtype=np.float16)
for j, (_, traces, _, _) in enumerate(data_loader((0,), (0,))):
    traces = traces[0][0]
    #traces_rws_only[j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, RWS_WINDOW]
    #traces_round_perm_only[j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, ROUND_PERM_WINDOW]
    for i in range(len(ROUND_WINDOWS)):
        traces_rounds_i_only[i, j * PER_DIVISION:j * PER_DIVISION + traces.shape[0]] = traces[:, ROUND_WINDOWS[i]]
    del traces

In [7]:
np.where(np.all(traces_rws_only == 0, axis=-1)), np.where(np.all(traces_round_perm_only == 0, axis=-1)), np.where(np.all(traces_rounds_i_only == 0, axis=-1))

(array([399999], dtype=int64),)

In [13]:
traces_rws_only = np.concatenate((traces_rws_only[:399999], traces_rws_only[400000:]), axis=0)
with open("traces_rws_only.pic", "wb") as w:
    pic.dump(traces_rws_only, w)

In [8]:
traces_round_perm_only = np.concatenate((traces_round_perm_only[:399999], traces_round_perm_only[400000:]), axis=0)
with open("traces_round_perm_only.pic", "wb") as w:
    pic.dump(traces_round_perm_only, w)

In [7]:
for i in range(len(ROUND_WINDOWS)):
    wout_empty_traces = np.concatenate((traces_rounds_i_only[i, :399999], traces_rounds_i_only[i, 400000:]), axis=0)
    with open(f"traces_round_{i}_adjusted_only.pic", "wb") as w:
        pic.dump(wout_empty_traces, w)

## Vizualisation

In [4]:
import pickle as pic
import plotly.express as px

folder = "leakage_points_bigdata"
all_plot_labels = []
try:
    with open(f"./{folder}/f_rws_perms_sost.pic", "rb") as r:
        sig_rws = pic.load(r) / (KEYROUND_WIDTH_B4 * (KEYROUND_WIDTH_B4 - 1) / 2)
    all_plot_labels.append("RWS perm")
except OSError:
    sig_rws = None
sig_masks_rws = []
for keyround_idx in range(KEYROUND_WIDTH_B4):
    for share_idx in range(NR_SHARES):
        try:
            with open(f"./{folder}/f_rws_masks_{keyround_idx}_{share_idx}_sost.pic", "rb") as r:
                sig_masks_rws.append(pic.load(r) / (len(KEY_ALPHABET) * (len(KEY_ALPHABET) - 1) / 2))
            all_plot_labels.append(f"RWS mask {keyround_idx} {share_idx}")
        except OSError:
            pass

try:
    with open(f"./{folder}/f_round_perms_sost.pic", "rb") as r:
        sig_round_perm = pic.load(r) / ((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4) * ((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4) - 1) / 2)
    all_plot_labels.append("Round perm")
except OSError:
    sig_round_perm = None
sig_copy_perms = []
sig_masks_perms = []
for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    try:
        with open(f"./{folder}/f_copy_perms_{round_idx}_sost.pic", "rb") as r:
            sig_copy_perms.append(pic.load(r) / (BLOCK_WIDTH_B4 * (BLOCK_WIDTH_B4 - 1) / 2))
        all_plot_labels.append(f"Copy perm {round_idx}")
    except OSError:
        pass

for round_idx in range(EARLIEST_ROUND, LATEST_ROUND):
    for block_idx in range(BLOCK_WIDTH_B4):
        for share_idx in range(NR_SHARES):
            try:
                with open(f"./{folder}/f_round_masks_{round_idx}_{block_idx}_{share_idx}_sost.pic", "rb") as r:
                    sig_masks_perms.append(pic.load(r) / (len(KEY_ALPHABET) * (len(KEY_ALPHABET) - 1) / 2))
                all_plot_labels.append(f"Round mask {round_idx} {block_idx} {share_idx}")
            except OSError:
                pass


In [ ]:
# Haar 3
trace_size = 25_000
for it in range(0, trace_size, trace_size // 4):
    everything = sig_masks_perms#([sig_rws] if sig_rws is not None else []) + sig_masks_rws + ([sig_round_perm] if sig_round_perm is not None else []) + sig_copy_perms + sig_masks_perms
    everything = [sost[it:it+trace_size // 4] for sost in everything]
    fig = px.line(y=everything)
    plot_labs = {f'wide_variable_{i}': l for i, l in enumerate(all_plot_labels)}
    fig.for_each_trace(lambda t: t.update(name = plot_labs[t.name], legendgroup = plot_labs[t.name], 
                                        hovertemplate = t.hovertemplate.replace(t.name, plot_labs[t.name])))
    fig.update_layout(yaxis_range=[-1,100])
    fig.show()

In [ ]:
# Haar 2
#trace_size = 50_000
#for it in range(0, trace_size, trace_size // 4):
#    everything = ([sig_rws] if sig_rws is not None else []) + sig_masks_rws + ([sig_round_perm] if sig_round_perm is not None else []) + sig_copy_perms + sig_masks_perms
#    everything = [sost[it:it+trace_size // 4] for sost in everything]
#    fig = px.line(y=everything)
#    plot_labs = {f'wide_variable_{i}': l for i, l in enumerate(all_plot_labels)}
#    fig.for_each_trace(lambda t: t.update(name = plot_labs[t.name], legendgroup = plot_labs[t.name], 
#                                        hovertemplate = t.hovertemplate.replace(t.name, plot_labs[t.name])))
#    fig.update_layout(yaxis_range=[-1,50])
#    fig.show()

#everything = ([sig_rws] if sig_rws is not None else []) + sig_masks_rws + ([sig_round_perm] if sig_round_perm is not None else []) + sig_copy_perms + sig_masks_perms
everything = [sig_round_perm] + sig_copy_perms + sig_masks_perms
fig = px.line(y=everything)
plot_labs = {f'wide_variable_{i}': l for i, l in enumerate(all_plot_labels)}
fig.for_each_trace(lambda t: t.update(name = plot_labs[t.name], legendgroup = plot_labs[t.name], 
                                    hovertemplate = t.hovertemplate.replace(t.name, plot_labs[t.name])))
fig.show()

In [ ]:
# Original traces
trace_size = 200_000
for it in range(0, trace_size, trace_size // 4):
    #everything = ([sig_rws] if sig_rws is not None else []) + sig_masks_rws + ([sig_round_perm] if sig_round_perm is not None else []) + sig_copy_perms + sig_masks_perms
    everything = [sig_rws] + sig_masks_rws
    everything = [sost[it:it+trace_size // 4] for sost in everything]
    fig = px.line(y=everything)
    plot_labs = {f'wide_variable_{i}': l for i, l in enumerate(all_plot_labels)}
    fig.for_each_trace(lambda t: t.update(name = plot_labs[t.name], legendgroup = plot_labs[t.name], 
                                        hovertemplate = t.hovertemplate.replace(t.name, plot_labs[t.name])))
    fig.show()
    break